Notes

- I installed snorkel from the redux branch into an environment running Python 3.6 -- 3.5 and 3.7 both didn't work for me. Skimming the git issues seems to indicate this is going to be fixed.
- Snorkel developers are working on a new release - so it may soon be better to install from master
- For data, I'm using the version of the Global Terrorism Database on Kaggle (https://www.kaggle.com/START-UMD/gtd/)

Beware of blogs, etc. about snorkel which may be helpful but often discuss outdated versions of the API!
- These tutorials WERE helpful, and then they got deleted: https://github.com/HazyResearch/snorkel/tree/redux/tutorials/workshop.
- Avoid these tutorials, which are outdated: https://github.com/HazyResearch/snorkel/blob/master/tutorials
- These tutorials seem to be up-to-date: https://github.com/snorkel-team/snorkel-tutorials/blob/master

In [136]:
import pandas as pd
import numpy as np
import snorkel
import re 

from sklearn.model_selection import train_test_split

In [200]:
terror_df = pd.read_csv('/Users/awhite/Documents/globalterrorismdb_0718dist.csv',
                       encoding = 'ISO-8859-1')

/anaconda3/envs/snorkel2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
terror_df.summary.head(20)

5     1/1/1970: Unknown African American assailants ...
7     1/2/1970: Unknown perpetrators detonated explo...
8     1/2/1970: Karl Armstrong, a member of the New ...
9     1/3/1970: Karl Armstrong, a member of the New ...
11    1/6/1970: Unknown perpetrators threw a Molotov...
13    1/9/1970: Unknown perpetrators set off a fireb...
14    1/9/1970:  The Armed Commandos of Liberation c...
17    1/12/1970: Unknown perpetrators threw a pipe b...
18    1/12/1970: Unknown perpetrators detonated a bo...
19    1/13/1970: Unknown perpetrators firebombed Fus...
20    1/14/1970: Suspected Black militants threw two...
22    1/17/1970: Three African Americans were suspec...
23    1/17/1970: Silas and Judith Bissell placed an ...
24    1/19/1970: Unknown perpetrators ignited a fire...
28    1/22/1970: Unknown perpetrators detonated an e...
29    1/25/1970: Unknown perpetrators burnt down a b...
30    1/25/1970: Responding to what authorities beli...
31    1/26/1970: Unknown African American assail

In [201]:
#drop items w no summary, be sure all summaries are strings
terror_df = terror_df.dropna(subset=['summary'])
#terror_df.summary = terror_df.summary.astype(str)
#terror_df.summary = terror_df.summary.str.lower()
len(terror_df)

115562

In [304]:
train, test = train_test_split(terror_df, test_size = 0.2, random_state = 0)

#snorkel calls for a separate development set
train, dev = train_test_split(train, test_size = 0.5, random_state = 0)

Y_dev = dev["suicide"].values
Y_test = test["suicide"].values

In [203]:
len(dev)

46225

Let's try to predict if an attack was a suicide attack - this seems easy.

In [317]:
pd.set_option("display.max_colwidth", 0)

terror_df[terror_df.suicide == 1].summary.sample(5, random_state=3)

76962    09/04/2004: A Police Academy in Kirkuk Iraq was attacked by a suicide bomber in a car, killing 20 and wounding 36. Tawhid and Jihad claimed responsibility for the attack.                                                                                                                                                                                                                                                                                       
76226    01/18/2004: A pickup truck loaded with 500 kilos of explosives exploded at the Assassin's Gate, the entrance to the main industrial center of Baghdad, Iraq, and also the United States' Military headquarters. Twenty-five were people killed and over 100 were injured in the attack, for which no group claimed responsibility.                                                                                                                               
77565    02/24/2005: An unidentified suicide car bomber wearing a police uniform t

In [316]:
terror_df[terror_df.suicide == 0].summary.sample(5, random_state=3)

112007    10/30/2012: Assailants detonated an explosive device at an electricity tower in the Ghitani area of Mach town, Balochistan province, Pakistan. There were no reported casualties; however, the targeted tower was partially damaged in the blast. No group claimed responsibility for the incident.                                                                                                               
156272    12/05/2015: An explosive device detonated as police personnel were attempting to defuse it in Waghaz district, Ghazni Province, Afghanistan. Two police officers were wounded in the blast. No group claimed responsibility for the incident.                                                                                                                                                                     
158353    01/31/2016: Explosive devices detonated targeting a military patrol in eastern Norte de Santander department, Colombia. Two soldiers were killed in the blast. No gr

In [145]:
from snorkel.labeling.apply import PandasLFApplier
from snorkel.labeling.lf import labeling_function
from snorkel.types import DataPoint

POS = 1
NEG = -1 
ABSTAIN = 0

In [268]:
dev.summary.str.contains("suicide attack").value_counts()

False    46093
True       132
Name: summary, dtype: int64

It seems snorkel isn't very forgiving about the way you write LFs. I tried a number of different ways for these basic examples - find a substring in text - and couldn't get them to work until I copied the approach in https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/01_spam_tutorial.ipynb. For instance `str.contains()` and `str.count()` didn't work. Maybe I was doing something wrong.

While there is some disagreement on this, snorkel developers generally suggest that many simple LFs are better than a few complex LFs.

In [292]:
@labeling_function()
def suicide_mentioned(x):
    return POS if "suicide" in x.summary.lower() else ABSTAIN

In [293]:
@labeling_function()
def suicide_attack(x):
    return POS if "suicide attack" in x.summary.lower() else ABSTAIN

In [294]:
@labeling_function()
def suicide_bomb(x):
    return POS if "suicide bomb" in x.summary.lower() else ABSTAIN

In [295]:
@labeling_function()
def unknown_perps(x):
    return NEG if "unknown perpetrator" in x.summary.lower() else ABSTAIN

In [319]:
@labeling_function()
def no_responsibility(x):
    return NEG if "no group claimed responsibility" in x.summary.lower() else ABSTAIN

In [320]:
applier = PandasLFApplier([suicide_mentioned,
                           suicide_attack,
                           suicide_bomb,
                          unknown_perps,
                          no_responsibility])

L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)






















  0%|          | 0/46224 [00:00<?, ?it/s]




















  0%|          | 1/46224 [00:00<10:39:22,  1.20it/s]




















  2%|▏         | 732/46224 [00:00<7:20:31,  1.72it/s]




















  3%|▎         | 1458/46224 [00:01<5:03:28,  2.46it/s]




















  5%|▍         | 2179/46224 [00:01<3:29:02,  3.51it/s]




















  6%|▋         | 2899/46224 [00:01<2:23:58,  5.02it/s]




















  8%|▊         | 3557/46224 [00:01<1:39:16,  7.16it/s]




















  9%|▉         | 4243/46224 [00:01<1:08:24, 10.23it/s]




















 11%|█         | 4922/46224 [00:01<47:08, 14.60it/s]  




















 12%|█▏        | 5636/46224 [00:01<32:27, 20.84it/s]




















 14%|█▎        | 6331/46224 [00:01<22:21, 29.74it/s]




















 15%|█▌        | 7026/46224 [00:01<15:24, 42.40it/s]




















 17%|█▋        | 7709/46224 [00:01<10:37, 60.41it/s]




















 18%|█▊        | 8413/46224 

 92%|█████████▏| 42325/46224 [00:06<00:00, 6801.29it/s]




















 93%|█████████▎| 43033/46224 [00:06<00:00, 6881.15it/s]




















 95%|█████████▍| 43722/46224 [00:06<00:00, 6879.23it/s]




















 96%|█████████▌| 44411/46224 [00:07<00:00, 6854.23it/s]




















 98%|█████████▊| 45135/46224 [00:07<00:00, 6964.45it/s]




















 99%|█████████▉| 45909/46224 [00:07<00:00, 7180.29it/s]




















100%|██████████| 46224/46224 [00:07<00:00, 6252.99it/s]




















  0%|          | 0/46225 [00:00<?, ?it/s]




















  0%|          | 1/46225 [00:00<9:45:03,  1.32it/s]




















  2%|▏         | 738/46225 [00:00<6:43:02,  1.88it/s]




















  3%|▎         | 1487/46225 [00:00<4:37:30,  2.69it/s]




















  5%|▍         | 2235/46225 [00:01<3:11:02,  3.84it/s]




















  6%|▋         | 2956/46225 [00:01<2:11:34,  5.48it/s]




















  8%|▊         | 3726/46225 [00:01<

 82%|████████▏ | 37719/46225 [00:05<00:01, 7579.46it/s]




















 83%|████████▎ | 38532/46225 [00:06<00:00, 7734.37it/s]




















 85%|████████▌ | 39343/46225 [00:06<00:00, 7841.68it/s]




















 87%|████████▋ | 40155/46225 [00:06<00:00, 7921.80it/s]




















 89%|████████▊ | 40966/46225 [00:06<00:00, 7974.82it/s]




















 90%|█████████ | 41765/46225 [00:06<00:00, 7953.70it/s]




















 92%|█████████▏| 42561/46225 [00:06<00:00, 7910.67it/s]




















 94%|█████████▍| 43375/46225 [00:06<00:00, 7976.08it/s]




















 96%|█████████▌| 44183/46225 [00:06<00:00, 8006.69it/s]




















 97%|█████████▋| 44985/46225 [00:06<00:00, 7791.68it/s]




















 99%|█████████▉| 45766/46225 [00:06<00:00, 7636.80it/s]




















100%|██████████| 46225/46225 [00:07<00:00, 6489.78it/s]

In [321]:
#The not-as-bad tutorial suggests LFAnalysis, which doesn't exist...
#lf_summary seems to be similar
from snorkel.labeling.analysis import lf_summary

lf_summary(L_dev)

,Polarity,Coverage,Overlaps,Conflicts
0,[1],0.053283,0.050449,0.025008
1,[1],0.002856,0.002856,0.001147
2,[1],0.047875,0.047875,0.023191
3,[-1],0.015857,0.001320,0.015857
4,[-1],0.657393,0.026198,0.657393


In [322]:
#If you supply gold labels, snorkel calculates empirical accuracy for you!
lf_summary(L_dev, Y=Y_dev)

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,[1],0.053283,0.050449,0.025008,2424,0,0.984166
1,[1],0.002856,0.002856,0.001147,126,0,0.954545
2,[1],0.047875,0.047875,0.023191,2193,0,0.990962
3,[-1],0.015857,0.001320,0.015857,3,0,0.000000
4,[-1],0.657393,0.026198,0.657393,1195,0,0.000000






















 33%|███▎      | 15444/46224 [00:21<00:14, 2141.30it/s]

So in theory we should stop here, because LFs by themselves give good accuracy. Or, if the description says suicide bomber...it's probably a suicide bomber. But let's work through to the end for fun.